<a href="https://colab.research.google.com/github/gmit22/LifeHack-2020/blob/master/Image_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
from sklearn.utils import shuffle
import imutils

In [2]:
from google.colab import drive
drive.mount('/content/drive')

file = open("/content/drive/My Drive/LifeHack/data.txt", "w")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
dir = os.path.join(os.getcwd(), "drive/My Drive/data")

In [3]:
def loadData(dir):

    images = []
    category = []
    file_dir = []

    for img in os.listdir(dir):

        if img.endswith(".png"):
        #images.append(cv2.imread(os.path.join(dir, img)))
            file_dir.append(os.path.join(dir, img))

            if img.startswith('NL'):      #assigning label to file according to          
                category.append(0)             # their type

            elif img.startswith('ca'):    
                category.append(1)

            elif img.startswith('Gl'):  
                category.append(2)

            elif img.startswith('Re'):          
                category.append(3)

    file_dir = np.array(file_dir)
    category = np.array(category, dtype = 'int32')  

    return shuffle(file_dir, category,random_state=817328462)


In [4]:
file_dir, category = loadData(dir)

In [5]:
print(len(file_dir))

601


In [6]:
def get_label(encoded_val):
      
      labels = {0:'normal', 1:'cataract', 2:'glaucoma', 3:'retina'}
      return labels[class_code]

In [7]:
def data_augment(file_dir, labels):

    img_set = []
    lbl_set = []

    for i in range(len(file_dir)):

        if i%100==0:
            print(i)
        label = labels[i]
        img = get_object(file_dir[i])

        img_b = img + 0.05*img
        img_d = img - 0.05*img

        flip_v_b = cv2.flip(img_b,0)
        flip_h_b = cv2.flip(img_b,1)

        flip_v_d = cv2.flip(img_d,0)
        flip_h_d = cv2.flip(img_d,1)

        flip_v = cv2.flip(img,0) #vertical flip
        flip_h = cv2.flip(img,1) #lr flip

        img_set.append(img)
        # img_set.append(img_b)
        # img_set.append(img_d)
        img_set.append(flip_v)
        img_set.append(flip_h)
        img_set.append(flip_v_b)
        img_set.append(flip_h_b)
        img_set.append(flip_v_d)
        img_set.append(flip_h_d)

        # lbl_set.append(label)
        # lbl_set.append(label)
        lbl_set.append(label)
        lbl_set.append(label)
        lbl_set.append(label)
        lbl_set.append(label)
        lbl_set.append(label)
        lbl_set.append(label)
        lbl_set.append(label)
    print('Done')
    return np.array(img_set),np.array(lbl_set)


In [8]:
def get_object(filename):

    image = cv2.imread(filename)
    im_size = 256
    
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) # convert 2 grayscale

    retval, threshold = cv2.threshold(gray,10,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(threshold,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) # find contours

    # ensure at least some circles were found
    if contours:

        contours = sorted(contours, key=cv2.contourArea, reverse=True) 

        #find the bounding rect
        x,y,w,h = cv2.boundingRect(contours[0])                  
        img = image[y:y+h,x:x+w]# crop image

        resize=cv2.resize(img,(im_size,im_size)) # resize to im_size X im_size size
        
        return resize

    return cv2.resize(image, (im_size,im_size))

In [9]:
X, y = data_augment(file_dir, category)

0
100
200
300
400
500
600
Done


In [10]:
print(len(X))

4207


Rotates the images 

In [11]:
import keras
y = keras.utils.to_categorical(y, 4)
X = X.astype('float32')
X /= 255

Using TensorFlow backend.


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [13]:
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import keras

Creating model using Keras

In [14]:
model = keras.Sequential()

#number of filters(patterns to be detected), window size 
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(256,256,3)))
model.add(Conv2D(32, (3,3), activation='relu'))

#Scales down, by discarding less useful data
model.add(MaxPooling2D(pool_size=(2, 2)))

#Prevents memorizing of datapoint sets
model.add(Dropout(0.3))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.summary()
##compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

#train the model
model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=20, #number of iterations
    validation_data=(X_test, y_test),
    shuffle=True     
)
# from pathlib import Path
# #save the neural network structure
# model_structure = model.to_json()
# f = Path("model_structure.json")
# f.write_text(model_structure)

# #Save the network's trained weights
# model.save_weights("model_weights.h5")


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 254, 254, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 127, 127, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 516128)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               66064512  
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

Model 2


In [27]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(4, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# train the model on the new data for a few epochs
model.fit(
    X_train,
    y_train,
    batch_size=16,
    epochs=50, #number of iterations
    validation_data=(X_test, y_test),
    shuffle=True
)

# # at this point, the top layers are well trained and we can start fine-tuning
# # convolutional layers from inception V3. We will freeze the bottom N layers
# # and train the remaining top layers.

# # let's visualize layer names and layer indices to see how many layers
# # we should freeze:
# for i, layer in enumerate(base_model.layers):
#    print(i, layer.name)

# # we chose to train the top 2 inception blocks, i.e. we will freeze
# # the first 249 layers and unfreeze the rest:
# for layer in model.layers[:249]:
#    layer.trainable = False
# for layer in model.layers[249:]:
#    layer.trainable = True

# # we need to recompile the model for these modifications to take effect
# # we use SGD with a low learning rate
# from tensorflow.keras.optimizers import SGD
# model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# # we train our model again (this time fine-tuning the top 2 inception blocks
# # alongside the top Dense layers
# model.fit(...)

Epoch 1/50
85/85 [==============================] - 15s 181ms/step - loss: 2.0221 - accuracy: 0.4771 - val_loss: 1.2565 - val_accuracy: 0.6430
Epoch 2/50
85/85 [==============================] - 14s 164ms/step - loss: 1.0797 - accuracy: 0.5754 - val_loss: 1.0438 - val_accuracy: 0.5898
Epoch 3/50
85/85 [==============================] - 14s 163ms/step - loss: 0.9528 - accuracy: 0.6176 - val_loss: 1.0967 - val_accuracy: 0.5987
Epoch 4/50
85/85 [==============================] - 14s 163ms/step - loss: 0.8913 - accuracy: 0.6220 - val_loss: 0.9514 - val_accuracy: 0.6319
Epoch 5/50
85/85 [==============================] - 14s 163ms/step - loss: 0.8338 - accuracy: 0.6723 - val_loss: 0.8784 - val_accuracy: 0.6386
Epoch 6/50
85/85 [==============================] - 14s 163ms/step - loss: 0.7916 - accuracy: 0.6812 - val_loss: 0.8818 - val_accuracy: 0.6386
Epoch 7/50
85/85 [==============================] - 14s 163ms/step - loss: 0.7572 - accuracy: 0.6857 - val_loss: 0.9570 - val_accuracy: 0.6652